# Text Generator

We are going to develop a text generator based on Markov Chains, and try different inputs and hiperparameters in order to achieve concrete outcomes that help us understand the possibilities of this technique. 

First we are going to model the problem with some classes in order to make the development easier.

In [ ]:
from numpy.random import choice
from collections import Counter
import matplotlib.pyplot as plt

class Token:
    def __init__(self, words):
        self.words = words
        self.count_following_tokens = 0
        self.following_tokens = {}

    def add_following_token(self, other_token):
        self.following_tokens[other_token] = self.following_tokens.get(other_token, 0) + 1
        self.count_following_tokens += 1
        
    def generate_next_token(self):
        next_tokens = list(self.following_tokens.keys())
        next_tokens_prob = list(map(lambda x: x / self.count_following_tokens, list(self.following_tokens.values())))
        if len(next_tokens) == 0:
            return None
        return choice(next_tokens, 1, p = next_tokens_prob)[0]
    
    def __repr__(self):
        return " ".join(list(map(str, self.words)))
    
    def __str__(self):
        return " ".join(list(map(str, self.words)))
    
    def __hash__(self):
        return hash("".join(list(map(str, self.words))))

    
    def __eq__(self, other):
        return type(other) == Token and self.words == other.words
    
    def __ne__(self, other):
        return type(other) != Token or self.words != other.words
        
        
class Word:
    def __init__(self, word):
        self.word = word
        
    def __repr__(self):
        return self.word
    
    def __str__(self):
        return self.word
    
    def __hash__(self):
        return hash(self.word)
    
    def __eq__(self, other):
        return type(other) == Word and self.word == other.word
    
    def __ne__(self, other):
        return type(other) != Word or self.word != other.word
        
class TextGenerator:
    def __init__(self, text, token_size):
        self.words_dict = {}
        self.words_list = []
        self.tokens_dict = {}
        self.tokens_list = []
        self.common_words = ["de", "la", "el", "en", "y", "del", "los", "a", "las", "que", "por", "se", "con", "un", "como", "una", "más", "es", "al", "su", "entre", "fue", "o", "también", "para", "son", "ha", "este", "desde", "lo", "sus", "si", "no", "uno", "sobre", "dos", "hasta", "durante", "the", "to", "and", "i", "you", "of", "we", "it", "that", "have", "they", "in", "going", "so", "is", "but", "were", "its", "be", "are", "for", "our", "this", "dont", "was", "do", "not", "with", "he", "im", "all", "what", "because", "want", "me", "said", "theyre", "on", "very", "about", "like", "if", "one", "them", "will", "at", "thats", "just", "now", "by", "when", "up", "these", "look", "been", "go", "from", "lot", "can", "know", "got", "many", "there", "my", "had", "good", "well", "make", "an", "or", "get", "think", "right", "out", "us", "really", "has", "hes", "as", "way", "mean", "much", "would", "over", "even", "who", "take", "youre", "ive", "than", "some", "never", "tell", "she", "doing", "time", "then", "say", "see", "other", "how", "their", "cant", "more", "didnt", "did", "things", "thing", "him", "where", "ever", "your", "into", "okay", "which", "something", "here", "need", "guy", "could", "ill", "why", "talk", "down", "done", "bad", "those", "actually", "weve", "let", "better", "ago", "new", "only", "came", "oh", "s", "apap", "ap", "unk"]
        self.available_characters = "abcdefghijklmnñopqrstuvwxyzáéíóú"
        self.text = text
        self.token_size = token_size
        
    def process_text(self):
        self.split_words()
        self.generate_tokens()
        self.calculate_probabilities()
        
    def split_words(self):
        words = self.text.split()
        for word in words:
            word = self.curate_word(word)
            if (word == ""):
                continue
            self.words_dict[word] = self.words_dict.get(word, Word(word))
            self.words_list.append(self.words_dict[word])

    def generate_tokens(self):
        for i in range(0, (len(self.words_list) // self.token_size) * self.token_size, self.token_size):
            previous_words = tuple([self.words_list[j] for j in range(i, i + self.token_size)])
            self.tokens_dict[previous_words] = self.tokens_dict.get(previous_words, Token(previous_words))
            self.tokens_list.append(self.tokens_dict[previous_words])

    def calculate_probabilities(self):
        for i in range(len(self.tokens_list) - 1):
            self.tokens_list[i].add_following_token(self.tokens_list[i + 1])

    def curate_word(self, word):
        word = word.lower()
        curated_word = ""
        for c in word:
            if c in self.available_characters:
                curated_word += c
        return curated_word
    
    def generate_text(self, length):
        generated_text = [self.pick_random_token()]
        for i in range(length - 1):
            next_token = generated_text[i].generate_next_token()
            if next_token == None:
                next_token = self.pick_random_token()
            generated_text.append(next_token)
        return ' '.join(list(map(str, generated_text)))
        
    def pick_random_token(self):
        return choice(list(self.tokens_dict.values()))
    
    def plot_most_important_words(self, n):
        counter = Counter(self.words_list)    
        words = sorted(counter.items(), key = lambda x: -x[1])
        most_important_words = []
        most_important_words_count = []
        for word in words:
            if word[0].word not in self.common_words:
                most_important_words.append(word[0].word)
                most_important_words_count.append(word[1])
                if (len(most_important_words) == n):
                    break
        fig = plt.figure(figsize=(30,10))
        plt.bar(most_important_words, most_important_words_count, orientation='vertical')
        fig.suptitle('Most important words', fontsize=40)
        plt.xticks(rotation=45, fontsize=20)
        
    def number_of_words(self):
        return len(self.words_list)
    
    def number_of_different_words(self):
        return len(self.words_dict)

In [ ]:
textGenerator = TextGenerator('a b a c a c a c d', 2)
textGenerator.process_text()
textGenerator.generate_text(10)

# Página de Argentina en Wikipedia
A continuación alimentaremos el generador de texto con la página de Argentina de Wikipedia en español.

In [ ]:
argentina_file = open("Argentina.txt")
argentina_content = " ".join(argentina_file.readlines())
textGeneratorArgentina = TextGenerator(argentina_content, 1)
textGeneratorArgentina.process_text()

number_of_words = textGeneratorArgentina.number_of_words()
number_of_different_words = textGeneratorArgentina.number_of_different_words()
print("Number of words: " + str(number_of_words) + ". Number of different words: " + str(number_of_different_words))

textGeneratorArgentina.plot_most_important_words(20)

## Generando texto teniendo en cuenta la palabra anterior

In [ ]:
textGeneratorArgentina = TextGenerator(argentina_content, 2)
textGeneratorArgentina.process_text()
textGeneratorArgentina.generate_text(15)

Algunas frases generadas:
* alvear el objeto de américa del país latinoamericano en la provincia de infraestructura entre otros
* espaciadas entre del precepto alberdiano de un país se vota por vicente lópez con los
* épico se encuentra en la renacionalización de gobernar es el instituto balseiro ubicado en bariloche

## Generando texto teniendo en cuenta las dos palabras anteriores

In [ ]:
textGeneratorArgentina = TextGenerator(argentina_content, 2)
textGeneratorArgentina.process_text()
textGeneratorArgentina.generate_text(10)

Algunas frases generadas:
* las letras del tango pero ha perdido buena parte de la alianza fue encabezada por el indec las provincias sus
* tanto públicas como privadas que se desarrollan también construye helicópteros maquinarias agrícolas produce el ciclo completo de la soja la
* ascendencia europea es menor y que constituye uno de los hombres obtuvieron el título más importante de la historia argentina

## Generando texto teniendo en cuenta las tres palabras anteriores

In [ ]:
textGeneratorArgentina = TextGenerator(argentina_content, 3)
textGeneratorArgentina.process_text()
textGeneratorArgentina.generate_text(10)

Algunas frases generadas:
* llamados comechingones resistieron con éxito la invasión incaica y se mantuvieron como señoríos independientes a
* español andino se fusiona con el dialecto de rioplatense la provincia de buenos aires con
* leche es muy importante consumiéndose alrededor de litros por persona por año de la existencia de grandes disponibilidades de leche se ha derivado un alto consumo de alimentos derivados como

# Página de Estados Unidos en Wikipedia
A continuación alimentaremos el generador de texto con la página de Estados Unidos en español.

In [ ]:
estados_unidos_file = open("EstadosUnidos.txt")
estados_unidos_content = " ".join(estados_unidos_file.readlines())
textGeneratorEstadosUnidos = TextGenerator(estados_unidos_content, 2)
textGeneratorEstadosUnidos.process_text()

number_of_words = textGeneratorEstadosUnidos.number_of_words()
number_of_different_words = textGeneratorEstadosUnidos.number_of_different_words()
print("Number of words: " + str(number_of_words) + ". Number of different words: " + str(number_of_different_words))

textGeneratorEstadosUnidos.plot_most_important_words(20)

## Generando texto teniendo en cuenta la palabra anterior

In [ ]:
textGeneratorEstadosUnidos = TextGenerator(estados_unidos_content, 1)
textGeneratorEstadosUnidos.process_text()
textGeneratorEstadosUnidos.generate_text(15)

Algunas frases generadas:
* reubicación de materiales y públicas de igual forma la música artículos principales carreras de julio
* cometieron el de influencia de los logros socioeconómicos de los aliados contra el estado islámico
* locales ocupaban el estado libre asociado con el feto ya que destacan autores como estado

## Generando texto teniendo en cuenta las dos palabras anteriores

In [ ]:
textGeneratorEstadosUnidos = TextGenerator(estados_unidos_content, 2)
textGeneratorEstadosUnidos.process_text()
textGeneratorEstadosUnidos.generate_text(10)

Algunas frases generadas:
* comercial estadounidense era de millones de afroamericanos que habían sido esclavos convirtiéndolos en ciudadanos y dándoles el derecho consuetudinario en
* mundo aunque en términos de gasto per cápita de toneladas de petróleo al año en salarios aunque su legalidad está
* transporte de mercancías por ferrocarril es muy importante relativamente pocas personas utilizan el transporte público para acudir al trabajo un

## Generando texto teniendo en cuenta las tres palabras anteriores

In [ ]:
textGeneratorEstadosUnidos = TextGenerator(estados_unidos_content, 3)
textGeneratorEstadosUnidos.process_text()
textGeneratorEstadosUnidos.generate_text(10)

Algunas frases generadas:
* y al presidente de los estados unidos donde se reúne el congreso estados unidos es una nación multicultural hogar de una amplia variedad de grupos étnicos tradiciones y valores aparte
* suelen emplearse de manera correcta la abreviatura ee uu estados unidos o la sigla eua estados unidos de américa aunque frecuente en español es incorrecto emplear la sigla inglesa usa
* es considerado un país megadiverso unas especies de plantas vasculares viven en los estados unidos cerca del de los universitarios asisten a colleges públicos como la universidad de virginia un

# Página de China en Wikipedia
A continuación alimentaremos el generador de texto con la página de China en español.

In [ ]:
china_file = open("China.txt")
china_content = " ".join(china_file.readlines())
textGeneratorChina = TextGenerator(china_content, 3)
textGeneratorChina.process_text()

number_of_words = textGeneratorChina.number_of_words()
number_of_different_words = textGeneratorChina.number_of_different_words()
print("Number of words: " + str(number_of_words) + ". Number of different words: " + str(number_of_different_words))

textGeneratorChina.plot_most_important_words(20)

## Generando texto teniendo en cuenta la palabra anterior

In [ ]:
textGeneratorChina = TextGenerator(china_content, 1)
textGeneratorChina.process_text()
textGeneratorChina.generate_text(15)

Algunas frases generadas:
* fin del arroz el segundo miembro de taiwán el kuomintang liderado por ejemplo de pekín
* incrementó los yuan y animales y los años atrás una vez más poblado del planeta
* septiembre de las dos hijos si lo que mantuvieron un el jengibre el muy criticado

## Generando texto teniendo en cuenta las dos palabras anteriores

In [ ]:
textGeneratorChina = TextGenerator(china_content, 2)
textGeneratorChina.process_text()
textGeneratorChina.generate_text(10)

Algunas frases generadas:
* área territorial después de la creación de la onu china tiene una tasa de mortalidad infantil es de por cada
* transitados en el mundo con y millones de chinos en las zonas rurales no tienen acceso a agua potable y
* ejército chino transporte el puente de donghai es uno de los mejores baloncestistas de china carreras de barcodragón un deporte

## Generando texto teniendo en cuenta las tres palabras anteriores

In [ ]:
textGeneratorChina = TextGenerator(china_content, 3)
textGeneratorChina.process_text()
textGeneratorChina.generate_text(10)

Algunas frases generadas:
* mundial y la retirada de sus tropas de china el partido comunista de china pcc cuyo poder está consagrado en la constitución la constitución de la república popular china están
* en términos de pib medido en paridad de poder adquisitivo y manteniéndose como la segunda potencia por pib nominal china es además el mayor exportador e importador de bienes y
* un país que tiene armas nucleares reconocidas china es considerada una potencia militar regional y una superpotencia militar emergente de acuerdo al informe de del departamento de defensa de estados

# Discursos de Trump en campaña presidencial

Discursos originales de Donald Trump en la campaña presidencial, en inglés. Se agregaron muchas palabras a la la lista de palabras comunes que no se consideraron importantes o representativas para el análisis del discurso

Extraídos de https://github.com/ryanmcdermott/trump-speeches/blob/master/speeches.txt

In [ ]:
trump_file = open("Trump.txt")
trump_content = " ".join(trump_file.readlines())
textGeneratorTrump = TextGenerator(trump_content, 1)
textGeneratorTrump.process_text()

number_of_words = textGeneratorTrump.number_of_words()
number_of_different_words = textGeneratorTrump.number_of_different_words()
print("Number of words: " + str(number_of_words) + ". Number of different words: " + str(number_of_different_words))

textGeneratorTrump.plot_most_important_words(30)

## Generando texto teniendo en cuenta la palabra anterior

In [ ]:
textGeneratorTrump = TextGenerator(trump_content, 1)
textGeneratorTrump.process_text()
textGeneratorTrump.generate_text(15)

Algunas frases generadas:
* global warming which are they are in by the radar nobody recognizes them iraq was
* upper income earned abroad in a picture because were going to bring back in the
* henry right now he spends million on something because of words radical islam is standing

## Generando texto teniendo en cuenta las dos palabras anteriores

In [ ]:
textGeneratorTrump = TextGenerator(trump_content, 2)
textGeneratorTrump.process_text()
textGeneratorTrump.generate_text(10)

Algunas frases generadas:
* any deals we all saw and we witnessed something that you could say well were going after hillary clinton she
* means less than zero and i said it very strongly in the meantime in michigan and other places and they
* though he was under pressure because theres nothing we have to get rid of the bullets went the opposite direction

## Generando texto teniendo en cuenta las tres palabras anteriores

In [ ]:
textGeneratorTrump = TextGenerator(trump_content, 3)
textGeneratorTrump.process_text()
textGeneratorTrump.generate_text(10)

Algunas frases generadas:
* four or five weeks ill tell you but they got access so they signed a pledge they will support i dont want their support it wont mean one vote than
* world take advantage of us both militarily and we dont win anymore we dont win anymore we dont win with trade we dont win we dont win at anything i
* extent is the power of weaponry its the power its the tremendous power you know years ago i mean it took them like years to build it so you know

# Discursos de Hillary Clinton en campaña presidencial

Discursos originales de Hillary Clinton en la campaña presidencial, en inglés. Se agregaron muchas palabras a la lista de palabras comunes que no se consideraron importantes o representativas para el análisis del discurso

Extraídos de https://votesmart.org/candidate/public-statements/55463/hillary-clinton?speechType=1#.XR_krJNKjOQ

In [ ]:
hillary_file = open("HillaryClinton.txt")
hillary_content = " ".join(hillary_file.readlines())
textGeneratorHillary = TextGenerator(hillary_content, 3)
textGeneratorHillary.process_text()

number_of_words = textGeneratorHillary.number_of_words()
number_of_different_words = textGeneratorHillary.number_of_different_words()
print("Number of words: " + str(number_of_words) + ". Number of different words: " + str(number_of_different_words))

textGeneratorHillary.plot_most_important_words(30)

## Generando texto teniendo en cuenta la palabra anterior

In [ ]:
textGeneratorHillary = TextGenerator(hillary_content, 1)
textGeneratorHillary.process_text()
textGeneratorHillary.generate_text(15)

Algunas frases generadas:
* youre wrong with we talk about illegal immigration on that brexits going on a statement
* reaping with honor of public schools and we might have more good middle class i
* shortchange her own health care of job is the hatefulness attacks one has the armed

## Generando texto teniendo en cuenta las dos palabras anteriores

In [ ]:
textGeneratorHillary = TextGenerator(hillary_content, 2)
textGeneratorHillary.process_text()
textGeneratorHillary.generate_text(10)

Algunas frases generadas:
* are within reach where families are supported streets are safe and yes we have created with all of this on
* work drive hundreds of campuses across the country attacks that disproportionately affect lowincome voters people of color students the elderly
* off donald trumps comments the textbook definition of a racist a person filled with hatred with an assault weapon we

## Generando texto teniendo en cuenta las tres palabras anteriores

In [ ]:
textGeneratorHillary = TextGenerator(hillary_content, 3)
textGeneratorHillary.process_text()
textGeneratorHillary.generate_text(10)

Algunas frases generadas:
* four or five weeks ill tell you but they got access so they signed a pledge they will support i dont want their support it wont mean one vote than
* world take advantage of us both militarily and we dont win anymore we dont win anymore we dont win with trade we dont win we dont win at anything i
* extent is the power of weaponry its the power its the tremendous power you know years ago i mean it took them like years to build it so you know

# Recopilación de noticias

Analizamos un gran dataset de noticias de diferentes medios de comunicación estadounidenses, en inglés.

Extraídos de https://catalog.ldc.upenn.edu/LDC97S44

In [ ]:
news_file = open("news.txt")
news_content = " ".join(news_file.readlines())
textGeneratorNews = TextGenerator(news_content, 3)
textGeneratorNews.process_text()

number_of_words = textGeneratorNews.number_of_words()
number_of_different_words = textGeneratorNews.number_of_different_words()
print("Number of words: " + str(number_of_words) + ". Number of different words: " + str(number_of_different_words))

textGeneratorNews.plot_most_important_words(30)

## Generando texto teniendo en cuenta la palabra anterior

In [ ]:
textGeneratorNews = TextGenerator(news_content, 1)
textGeneratorNews.process_text()
textGeneratorNews.generate_text(15)

Algunas frases generadas:
* plunk down threats as a search platforms microsoft will need to make sure some jewish
* syndicate was the chinese officials said on the principle of plotting a third avenue stores
* nonsectarian cards and international boxing final quot ordeal ugly and explore the whole truth about

## Generando texto teniendo en cuenta las dos palabras anteriores

In [ ]:
textGeneratorNews = TextGenerator(news_content, 2)
textGeneratorNews.process_text()
textGeneratorNews.generate_text(10)

Algunas frases generadas:
* by stepping up quota compliance to thwart a takeover investors still got to celebrate a historic development in the computing
* sunday liberals celebrate as ukraine waits poll result reuters reuters us house has approved a stent inserted wednesday into an
* blade a spanish supercomputer built by china and india the shift would wreck mideast peace efforts the bush administration dramatically

## Generando texto teniendo en cuenta las tres palabras anteriores

In [ ]:
textGeneratorNews = TextGenerator(news_content, 3)
textGeneratorNews.process_text()
textGeneratorNews.generate_text(10)

Algunas frases generadas:
* enough by kristen philipkoski got antinuke pills probably not potassium iodide pills could ward off cancer in the event of a nuclear accident but many states have refused to take
* are more exposed than ever to what could be a final step on the long road to bringing the global pact into force dogs sniff out bladder cancer dogs can
* league arsenal dropped five points behind chelsea in the summer the dutch side has scored goals in the final minutes to beat hamburg sv jol wants fulltime job at tottenham